## Implement a Wiki Question Answering with Tools Application
- Learning How to Use Langchain Agent to Implement an ALM Application
    - Reasoning: Build-in Prompt Template
    - Action: Tool Desciption 
        - Wiki API
        - Calculator

<img src="./imgs/QA_with_tool.PNG" alt="QA with Tools" width="600" height="300" style="float: left; margin-right: 10px;">

In [6]:
# !pip install langchain
# !pip install openai
# !pip install wikipedia

In [17]:
# %set_env OPENAI_API_KEY=OPENAI_API_KEY

In [5]:
import os

import langchain
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.utilities import WikipediaAPIWrapper
from langchain.agents.tools import Tool
from langchain import LLMMathChain
from langchain.agents import initialize_agent, AgentType

In [6]:
### set openai api key ###
# os.environ["OPENAI_API_KEY"] = ### put your own key here
###

## ALM = LLM + Reasoning + Action

### 1. Prepare LLM
#### [Ref: Chat mode LLM model from Langchain](https://python.langchain.com/docs/modules/model_io/models/chat/)
#### [Ref: LLM model from Langchain](https://python.langchain.com/docs/modules/model_io/models/llms/)

In [7]:
### openai model: GPT3.5-Turbo
chat_llm = ChatOpenAI(temperature=0)

### another openai model: text-davinci-003
llm = OpenAI(temperature=0)

### 2. Prepare tools
#### [Ref: Wikipedia tool from Langchain](https://python.langchain.com/docs/modules/agents/tools/integrations/wikipedia)
#### [Ref: Math tool from Langchain](https://python.langchain.com/docs/modules/chains/additional/llm_math)
#### [Ref: Tools from Langchain ](https://python.langchain.com/docs/modules/agents/tools/how_to/custom_tools)

In [8]:
### tool for Wikipedia
wikipedia = WikipediaAPIWrapper()

### for math tool
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

In [9]:
### 
# Define tools you need and relatived information for agent(LLM) to read
# 1. Prepare tool name
# 2. Prepare tool function
# 3. Prepare tool description  
###

###
# method 1: 
###

tools = [
    Tool(
        name = "Wikipedia",
        func=wikipedia.run,
        description="""A wrapper around Wikipedia.\ 
        Useful for when you need to answer general questions about people, places, companies, facts, historical events,\ 
        or other subjects. Input should be a search query."""
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
]

In [10]:
###
# method 2: 
###
# tools = load_tools(["llm-math","wikipedia"], llm=llm)

### 3. Prepare build-in component: Agent, which have build-in reasoning prompt template
#### [Ref: build-in reasoning prompt template - agent, from Langchain](https://python.langchain.com/docs/modules/agents/agent_types/react)

In [13]:
agent = initialize_agent(tools, llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

### 4. Prepare your input  question and run 

#### Question1. Math only

In [14]:
question1 = "What is the 25% of 300?"

agent.run(question1)



> Entering new  chain...
Thought: I need to calculate the 25% of 300
Action:
```
{
  "action": "Calculator",
  "action_input": "25% of 300"
}
```


> Entering new  chain...
25% of 300```text
300 * 0.25
```
...numexpr.evaluate("300 * 0.25")...

Answer: 75.0
> Finished chain.

Observation: Answer: 75.0
Thought: I now know the final answer
Final Answer: 75.0

> Finished chain.


'75.0'

#### Question2. Wiki Search only

In [15]:
question2 = "Who is Leo DiCaprio's girlfriend?"

agent.run(question2)



> Entering new  chain...
Thought: I need to find out who Leo DiCaprio's girlfriend is.
Action:
```
{
  "action": "Wikipedia",
  "action_input": "Leo DiCaprio's girlfriend"
}
```

Observation: Page: Leonardo DiCaprio
Summary: Leonardo Wilhelm DiCaprio (, ; Italian: [diˈkaːprjo]; born November 11, 1974) is an American actor and film producer. Known for his work in biographical and period films, he is the recipient of numerous accolades, including an Academy Award, a British Academy Film Award and three Golden Globe Awards. As of 2019, his films have grossed over $7.2 billion worldwide, and he has been placed eight times in annual rankings of the world's highest-paid actors.
Born in Los Angeles, DiCaprio began his career in the late 1980s by appearing in television commercials. In the early 1990s, he had recurring roles in various television shows, such as the sitcom Parenthood, and had his first major film part as author Tobias Wolff in This Boy's Life (1993). He received critical accl

"Leonardo DiCaprio's girlfriend is Camila Morrone."

#### Question3. Wiki Search and calculate 

In [16]:
question3 = "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"

agent.run(question3)



> Entering new  chain...
Thought: I need to find out who Leo DiCaprio's girlfriend is and then calculate her current age raised to the 0.43 power.

Action:
```
{
  "action": "Wikipedia",
  "action_input": "Leo DiCaprio's girlfriend"
}
```

Observation: Page: Leonardo DiCaprio
Summary: Leonardo Wilhelm DiCaprio (, ; Italian: [diˈkaːprjo]; born November 11, 1974) is an American actor and film producer. Known for his work in biographical and period films, he is the recipient of numerous accolades, including an Academy Award, a British Academy Film Award and three Golden Globe Awards. As of 2019, his films have grossed over $7.2 billion worldwide, and he has been placed eight times in annual rankings of the world's highest-paid actors.
Born in Los Angeles, DiCaprio began his career in the late 1980s by appearing in television commercials. In the early 1990s, he had recurring roles in various television shows, such as the sitcom Parenthood, and had his first major film part as author Tobi

"The current age of Leo DiCaprio's girlfriend, Camila Morrone, raised to the 0.43 power is 3.9218486893172186."